In [1]:
from collections import defaultdict
import pandas as pd
import MySQLdb
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np

'''
    We're using the Operon model here built from the e.coli 
    baceterium's integernic distances to predict operon membership
    between genes inside of the agro bacterium genome
'''

In [2]:
# recreate the operon model
pos_ctrl, neg_ctrl = [], []

# load the negative and positive controls first
with open('data/pos_ctrl.txt') as f:
    f = f.read()
    pos_ctrl = [int(num) for num in f.strip('\n').split('\t')]
with open('data/neg_ctrl.txt') as f:
    f = f.read()
    neg_ctrl = [int(num) for num in f.strip('\n').split('\t')]
    
# generate the log liklihood out of the positive and negative control
LL_h1 = gaussian_kde(pos_ctrl)
LL_h0 = gaussian_kde(neg_ctrl)

# build our model
def model(x):
    num = LL_h1(x)*0.60
    den = LL_h0(x)*0.40 + num
    
    return (num/den)

# the threshld for classifying operon membership is 0.60
threshold = 0.60

In [3]:
# Set up the python binding for MySQL db
db = MySQLdb.connect(host='localhost',
                     user='root',
                     passwd='REDACTED',
                     db='REDACTED')
curr = db.cursor()

In [4]:
# get all the genes from each of the replicons and order them by their left position of their first exons. 
# Then get the distances between each of the gene.
sql = 'SELECT g.gene_id,e.left_position,e.right_position,g.strand FROM genes g INNER JOIN exons e USING(gene_id) INNER JOIN replicons r USING(replicon_id) WHERE replicon_id = XXX ORDER BY e.left_position ASC'

predictions = []

# the replicoins of the agro bacterium we'll sift through
replicons = [2,3,4,5]

# iterate by replicion within agro bacterium
for replicon in replicons:
    # modify the sql statement with the right replicon id
    replicon_sql = sql.replace('XXX', str(replicon))

    # execute the sql statement
    curr.execute(replicon_sql)
    result = curr.fetchall()
    
    # make the results more nicer to access
    result = [{'left': left, 'right': right, 'strand': strand, 'gene_id': gene_id} for (gene_id, left, right, strand) in result]
    result = sorted(result, key=lambda x: x['left'])
    
    # get the distance between every pair of two adjacent genes
    for i in xrange(len(result)-1): 
        geneA = result[i]
        geneB = result[i+1]
        
        # make sure they're within the same strand
        if geneA['strand'] != geneB['strand']:
            continue
    
        dist = geneB['left'] - geneA['right'] + 1
        
        pred = 'P' if model(dist)[0] >= threshold else 'N'
        
        predictions.append({
                'gid_1': geneA['gene_id'],
                'gid_2': geneB['gene_id'],
                'dist': dist,
                'strand': geneA['strand'],
                'pred': pred,
                'replicion_id': replicon
            })

In [5]:
# Sample output of Agro Bacterium operon membership prediction
# between two genes from the Agro Bacterium genome
print '\t'.join(map(str, [dt[0] for dt in predictions[0].items()]))
for prediction in predictions[:10]:
    print '\t'.join(map(str, [dt[1] for dt in prediction.items()]))

dist	replicion_id	pred	gid_1	gid_2	strand
32	2	P	4320	4321	1
-7	2	P	4321	4322	1
0	2	P	4322	4323	1
-7	2	P	4323	4324	1
100	2	N	4325	4326	-1
-3	2	P	4327	4328	1
-3	2	P	4328	4329	1
-3	2	P	4329	4330	1
156	2	N	4330	4331	1
126	2	N	4333	4334	1
